In [1]:
!pip install gradio
!pip install keybert

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 86.0/86.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------- 238.4/238.4 kB 14.3 MB/s eta 0:00:00
     ---------------------------------------- 51.1/51.1 kB ? eta 0:00:00
     ---------------------------------------- 1.1/1.1 MB 23.9 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.7.0-py3-none-any.whl size=23776 sha256=3734a1fa6aedfc208988df8011e8ee702d857ba04f4c0417258ef2604ff2424d
  Stored in directory: c:\users\gabri\appdata\local\pip\cache\wheels\68\aa\41\82025d89b0eb97484c9ac7d527abf596765c41733af79f86b0
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=31b288559f78126302be8c0d842296dbb58d154230ade9db865f690afe860ef5
  Stored in directory:

In [1]:
# Imports
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
import numpy as np
from PIL import Image
import gradio as gr
import sys
import openai
sys.path.append("../")
import speech_recognition as sr
import pygame
from pygame import mixer
import cv2

C:\Users\gabri\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\gabri\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\gabri\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


pygame 2.1.2 (SDL 2.0.18, Python 3.9.15)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
from putting_it_together import WaifuObj

In [3]:
# Audio recognizer
r = sr.Recognizer()

In [4]:
# Path to the custom audio model
audio_model_path = "../Audio_Generation/Generation_Scripts/saved_models/default"

# Path to the custom audio data
audio_data_path = "../Audio_Generation/Generation_Scripts/data/albedo"

# Path to the custom model to load in
custom_model_path = "../Finetuning/outputs/r/"

In [5]:
# The initial summary is initially a basic prompt telling GPT-3 who it is
initial_summ = "You are my female waifu girlfriend who loves me."\
# The initial prompt tells GPT-3 how to respond
initial_prompt = "Me: Hi\nYou: Hello\n\n"\
    "Me: How are you?\nYou: Good. How are you?\n\n"\
    "Me: I'm good.\nYou: Nice to meet you.\n\n"

In [6]:
# Setup function to setup the environment
memory_file = "config_file.json"
Module = WaifuObj(initial_summ, initial_prompt, False, audio_model_path, audio_data_path, custom_model_path, memory_file)

Initializing image model...


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Image model initialized!
Initializing custom text model
Custom text model initialized!
Not loading custom audio model


In [ ]:
def build_img_prompt(text, settings, characteristics):
    # Get the summary and sentiment
    sent = get_sent(text)
    summary = get_summ(text)
    
    # Create the image prompt
    # settings = "1girl, very wide shot, simple background, solo focus, female focus, looking at viewer, ratio:16:9, detailed"
    # characteristics = "waifu, female, brown hair, blue eyes, sidelocks, slight blush, fox ears"
    # sent = "furious"
    # summary = "'I hope get know better' to viewer"
    img_prompt = f"{settings} {characteristics} {','+sent if len(sent)!=0 else ''}, {summary}"
    return img_prompt


def text_to_image(settings, characteristics, guidance_scale, text):
    # Get the image prompt
    img_prompt = Module.build_img_prompt(text, settings, characteristics)
    
    # Get the image
    with Module.suppress_stdout():
        with autocast("cuda"):
            image = Module.imgGen(img_prompt, guidance_scale=guidance_scale)["images"]
            
    
    return image

# Overall function to generate audio,
# text, and images
def generate(custom_audio, custom_model, settings, characteristics, guidance_scale, text, audio_pth, GPT_key):
    global prompt
    global last_image
    global last_text
    
    # Get the audio if there is any
    if audio_pth:
        # Open the wav file and read in the data
        # Get the audio data
        audio = sr.AudioFile(audio_pth)
        with audio as source:
            audio = r.record(source)
        
        text = audio_to_text(audio)
    
    # Add the text to the current prompt
    Module.cur_prompt += f"Me: {text}\n"
    
    # Get the response
    if custom_model == True:
        ret_text = Module.get_response()
    else:
        try:
            ret_text = ret_text = Module.get_response(GPT_key)
        except:
            gr.Error("GPT key is either invalid or not given")
            return last_image, "Error: GPT key is either invalid or not given."
    
    # Create audio and image for the returned text
    if len(ret_text) > 3:
        # Create the audio clip
        mixer.stop()
        mixer.music.unload()
        Module.create_audio(ret_text, custom_audio)

        # Play the audio
        try:
            mixer.music.load('tmp.mp3')
            mixer.music.play()
        except Module.pygame.error:
            s = mixer.Sound('tmp.mp3')
            s.play()
            
        # Generate an image from the text
        image = text_to_image(settings, characteristics, guidance_scale, ret_text)[0]
        
        # Save the image and text
        last_image = image
        last_text = ret_text
        
        # Return the image
        return image, ret_text
    
# Transcribes audio to text
def audio_to_text(audio):
    try:
        text = r.recognize_google(audio)
    except sr.UnknownValueError:
        text = "" # Default to nothing
            
    return text


def audio_auto_submit(custom_audio, custom_model, settings, characteristics, guidance_scale, text, audio_pth, GPT_key):
    if audio_pth != None:
        return generate(custom_audio, custom_model, settings, characteristics, guidance_scale, text, audio_pth, GPT_key)
    global last_image
    global last_text
    return last_image, last_text

# Incase of an error, these can be used
global last_image
global last_text
last_image = np.zeros((50, 50, 3))
last_text = "Error"

# Initialize the audio mixer
mixer.init()
mixer.music.unload()


def load_mem(filename):
    try:
        Module.load_memory(filename)
        return "Success!"
    except:
        return "Fail! File does not exist or is in incorrect format"


interface = gr.Blocks()
with interface:
    with gr.Tabs():
        with gr.TabItem("Generation"):
            # Talking to the AI
            with gr.Tabs():
                with gr.TabItem("Voice-based Chat"):
                    audio = gr.Audio(source="microphone", type="filepath", label="Response", live=True)
                with gr.TabItem("Text-based Chat"):
                    text = gr.Textbox(label="Text", value="I love you!", interactive=True)
            response = gr.Textbox(label="Response", value="", interactive=False)
                

            # Note gallery expects a 3-D array: (L, W, 3)
            gallery = gr.Image(label="Generated images", show_label=False)\
                .style(height=512)

            # Button to generate an image
            btn = gr.Button("Generate")
            
            
        with gr.TabItem("Settings"):
            # Switched for which model to use
            custom_model = gr.Checkbox(value=True, label="Use custom chat model? (False to use GPT, True to use custom model)")
            GPT_key_ = gr.Textbox(label="Key to use GPT-3 (if using GPT-3)\nNote: If you don't have one go here: https://elephas.app/blog/how-to-create-openai-api-keys-cl5c4f21d281431po7k8fgyol0", value="", interactive=True)
            custom_audio = gr.Checkbox(value=False, label="Use custom audio model?")

            # Settings for the image
            settings = gr.Textbox(label="Settings", value= "1girl, very wide shot, simple background, solo focus, female focus, looking at viewer, ratio:16:9, detailed", interactive=True)
            characteristics = gr.Textbox(label="Characteristics", value="waifu, female, brown hair, blue eyes, sidelocks, slight blush", interactive=True)
            guidance_scale = gr.Number(label="Guidance value - Tradeoff between creativity and image fidelity (greater than 1.0)", value=10.0, interactive=True, precision=1)
            
            # Load in some memory
            with gr.Column():
                mem_file = gr.Textbox(label="Memory File", value= "", interactive=True)
                with gr.Row():
                    mem_load_btn = gr.Button("Load memory file")
                    mem_file_success = gr.Textbox(label="Was the load successful?", value= "", interactive=False)
                    mem_load_btn.click(fn=load_mem, inputs=[mem_file], outputs=[mem_file_success])
            
            
        audio.change(audio_auto_submit, inputs=[custom_audio, custom_model, settings, characteristics, guidance_scale, text, audio, GPT_key_], outputs=[gallery, response])
        btn.click(fn=generate, inputs=[custom_audio, custom_model, settings, characteristics, guidance_scale, text, audio, GPT_key_], outputs=[gallery, response])
        text.submit(fn=generate, inputs=[custom_audio, custom_model, settings, characteristics, guidance_scale, text, audio, GPT_key_], outputs=[gallery, response])

interface.launch(debug=False, share=False)

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


  0%|          | 0/50 [00:00<?, ?it/s]